In [3]:
## 공공데이터 오픈 API이용해서 나라별 경보 정보 추출 

import requests
data={}
name=[]
level=[]
engname=[]
for i in range(1,21):
    url = 'http://apis.data.go.kr/1262000/TravelAlarmService2/getTravelAlarmList2'
    params ={'serviceKey' : 'txmwHHopJpj8GF/H9ZxtxAlWe4djLJZXzhcvIgV6dmKh1eJaqmDw29KiY55/B53y/oOFLiQI20qjZAVLNpUwvw==','pageNo' : f'{i}' }
    response = requests.get(url, params=params).json()['data'] 
    for j in range(len(response)):
        re=response[j]
        name.append(re['country_nm'])
        engname.append(re['country_iso_alp2'])
        level.append(re['alarm_lvl'])
    data['country']=name
    data['eng']=engname
    data['level']=level
data
    

{'country': ['가나',
  '가봉',
  '가이아나',
  '감비아',
  '과테말라',
  '교황청',
  '그레나다',
  '그리스',
  '기니',
  '기니비사우',
  '나미비아',
  '나우루',
  '나이지리아',
  '남수단',
  '남아프리카공화국',
  '네덜란드',
  '네팔연방',
  '노르웨이',
  '뉴질랜드',
  '니우에',
  '니제르',
  '니카라과',
  '대만',
  '덴마크',
  '도미니카공화국',
  '도미니카연방',
  '독일',
  '동티모르',
  '라오스',
  '라이베리아',
  '라트비아',
  '러시아',
  '레바논',
  '레소토',
  '루마니아',
  '룩셈부르크',
  '르완다',
  '리비아',
  '리투아니아',
  '리히텐슈타인',
  '마다가스카르',
  '마셜제도',
  '마이크로네시아연방',
  '말라위',
  '말레이시아',
  '말리',
  '멕시코',
  '모나코',
  '모로코',
  '모리셔스',
  '모리타니아',
  '모잠비크',
  '몬테네그로',
  '몰도바',
  '몰디브',
  '몰타',
  '몽골',
  '미얀마',
  '미합중국',
  '바누아투',
  '바레인',
  '바베이도스',
  '바하마',
  '방글라데시',
  '베냉',
  '베네수엘라볼리바르',
  '베트남',
  '벨기에',
  '벨라루스',
  '벨리즈',
  '보스니아헤르체고비나',
  '보츠와나',
  '볼리비아',
  '부룬디',
  '부르키나파소',
  '부탄',
  '북마케도니아',
  '불가리아',
  '브라질',
  '브루나이',
  '사모아',
  '사우디아라비아',
  '사이프러스',
  '산마리노',
  '상투메프린시페',
  '세네갈',
  '세르비아',
  '세이셸',
  '세인트루시아',
  '세인트빈센트그레나딘',
  '세인트키츠네비스',
  '소말리아',
  '솔로몬제도',
  '수단',
  '수리남',
  '스리랑카',
  '스웨덴',
  '스위스',
  

In [12]:
import pandas as pd
lf=pd.DataFrame(data)
lf


,country,eng,level
0,가나,GH,2.0
1,가봉,GA,1.0
2,가이아나,GY,2.0
3,감비아,GM,2.0
4,과테말라,GT,2.0
...,...,...,...
192,핀란드,FI,1.0
193,필리핀,PH,4.0
194,헝가리,HU,2.0
195,호주,AU,1.0


In [13]:
lf.iloc[58,0]='미국'    #미합중국 -> 미국으로 변경
lf=lf.fillna('2.5')     #공백=특별경보지역 ->임의의 수 2.5로 변경
lf['level']=lf['level'].astype('float') # 데이터 타입 통일
lf

,country,eng,level
0,가나,GH,2.0
1,가봉,GA,1.0
2,가이아나,GY,2.0
3,감비아,GM,2.0
4,과테말라,GT,2.0
...,...,...,...
192,핀란드,FI,1.0
193,필리핀,PH,4.0
194,헝가리,HU,2.0
195,호주,AU,1.0


In [14]:
lf=lf.loc[lf['level']<3.0,['country','eng','level']]  #1-3단계만 필터링
lf

,country,eng,level
0,가나,GH,2.0
1,가봉,GA,1.0
2,가이아나,GY,2.0
3,감비아,GM,2.0
4,과테말라,GT,2.0
...,...,...,...
191,피지,FJ,2.0
192,핀란드,FI,1.0
194,헝가리,HU,2.0
195,호주,AU,1.0


In [17]:
lf.to_excel('data/나라별.xlsx',index=False)

In [19]:
sheets=['Asia','Europe','Africa','America','Oceania']
result=pd.DataFrame()
df=pd.DataFrame()
for sheet in sheets:
    tr=pd.read_excel('data/목적지별 해외출국.xlsx',header=1,sheet_name=sheet)
    tr=tr.iloc[229:,4::2]
    series=tr.sum()
    df2=pd.DataFrame(series)
    df=pd.concat((df,df2),sort=False)
    
df['country']=df.index
df.columns=['total','country']
df['total']=df['total'].astype('int32')
df=df[['country','total']]
df

C:\Users\djagk\AppData\Local\Temp\ipykernel_15260\2203721075.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  series=tr.sum()
C:\Users\djagk\AppData\Local\Temp\ipykernel_15260\2203721075.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  series=tr.sum()


,country,total
일본,일본,58184349
중국,중국,71419895
베트남,베트남,19240727
태국,태국,21743354
필리핀,필리핀,16616347
...,...,...
북마리아나(사이판),북마리아나(사이판),2277481
괌(미국령),괌(미국령),3295822
뉴질랜드,뉴질랜드,1289970
팔라우,팔라우,210491


In [20]:
df.to_excel('data/출국자수.xlsx',index=False)


In [24]:
import pandas as pd
level=pd.read_excel('data/나라별.xlsx')
level
count=pd.read_excel('data/출국자수.xlsx')
df=count.sort_values('total',ascending=False).reset_index(drop=True)


In [25]:
count['total']=count['total'].astype('int32')

In [26]:
result=pd.merge(left=level,right=count,how='left',on='country')
result=result.fillna(0)
result['rank']=result['total'].rank(method='max',ascending=False).astype('int32')
result=result.reset_index(drop=True)
result=result.sort_values('rank')

In [27]:
result

,country,eng,level,total,rank
106,중국,CN,2.5,71419895.0,1
46,미국,US,2.0,22902654.0,2
51,베트남,VN,2.0,19240727.0,3
139,홍콩,HK,2.5,17078247.0,4
19,대만,TW,2.5,8159426.0,5
...,...,...,...,...,...
41,몬테네그로,ME,2.0,0.0,140
39,모나코,MC,2.0,0.0,140
37,말라위,MW,2.5,0.0,140
82,아르헨티나,AR,2.0,0.0,140


In [30]:
result.to_excel('data/나라별경보.xlsx',index=False)